# Dimensionality reduction

- PCA & UMAP 
- Clustering using spectral clustering

### Load in the data from online location

### PCA

### UMAP

### PCA -> Spectral clustering

### UMAP -> Spectral clustering